# TM10007 Assignment template

In [ ]:
# Run this to use from colab environment
# %pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

In [ ]:
# %pip install -U scikit-learn
# %pip install mlxtend

# Extentions used in assignment

In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from worcliver.load_data import load_data, load_Ft_set, load_D_set, second_split, load_Tr_set, load_Va_set

from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.feature_selection import VarianceThreshold, RFE
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve
from scipy.stats import randint
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

import importlib
import function_files.self_made

importlib.reload(function_files.self_made)

from function_files.self_made import classifiers


# Functions used in assignment
There are multiple functions used for this assignment. Every function is explained in the module of the function itself.

In [ ]:
def return_X_y(df):
    '''
    Returns dataframe y_d including the labels, and a seperate dataframe X_d including the feature information.
    '''
    y_d = df.pop('label').to_frame()
    X_d = df

    return X_d, y_d


def preprocessing(feature_df):
    '''
    This function preprocesses the features by removing zero-variance, zero-filled,
    and nan-filled features from the dataset.
    '''

    filter = VarianceThreshold(threshold=0.0)
    df_filter1 = filter.fit_transform(feature_df)
    features_kept = filter.get_support()
    names = feature_df.columns[features_kept]
    df_filter2 = pd.DataFrame(df_filter1, columns=names)
    zero_var = pd.DataFrame(feature_df.columns[~features_kept], columns=["removed_features"])
    zero_var_removed = feature_df.columns[~features_kept].tolist()

    # remove zero-filled features (if >80% is zero)
    zero_percents = (df_filter2 == 0).sum() / len(df_filter2) * 100
    threshold = 80
    nonzero_cols = df_filter2.columns[zero_percents <= threshold]
    zero_cols = df_filter2.columns[zero_percents > threshold]
    df_filtered = pd.DataFrame(df_filter2, columns=nonzero_cols)
    high_zero_frac_removed = zero_cols.tolist()

    print(f"Number of removed columns due to zero-variance: {len(zero_var)}")
    print(f"Number of removed non-zero-variance columns due to fraction zero > 80%: {len(zero_cols)}")
    print(f'Remaining number of features after preprocessing: {df_filtered.shape[1]}')

    removed_features = zero_var_removed + high_zero_frac_removed

    return df_filtered, removed_features


def outliers(X_filtered):
    means = X_filtered.mean()
    stds = X_filtered.std()
    threshold = 3
    for feature in X_filtered.columns:

        # Calculate the lower and upper bounds for the feature
        lower_bound = means[feature] - (stds[feature] * threshold)
        upper_bound = means[feature] + (stds[feature] * threshold)

        # Replace outliers with upper and lower bound
        X_filtered.loc[X_filtered[feature] < lower_bound, feature] = lower_bound
        X_filtered.loc[X_filtered[feature] > upper_bound, feature] = upper_bound
    
    return X_filtered


def normalize_standard_v1(column):
    '''
    This function will normalize a panda Series using the standard scaling.
    It will return a new column with normalized numbers with a mean of 0 and a standard deviation of 1. 
    Assumed is that the data has a normal distribution. To apply this to a dataframe
    this function needs to be applied to each column (which can be done with the use of .apply())
    '''
    feat2 = column.to_numpy()
    standard_scale = StandardScaler()
    feat_scaled2 = standard_scale.fit_transform(feat2.reshape(-1, 1))
    return pd.Series(feat_scaled2.flatten(), index=column.index)


def plt_tsne(df_features, labels):
    '''
    Using this function will return a 2 dimensional t-SNE plot.
    May be used as visualisation and possible feature extraction.
    '''

    labels = labels['label'].replace({'benign': 0, 'malignant': 1})
    labels = labels.to_numpy()
    df_features = df_features.to_numpy()

    # Perform TSNE
    tsne = TSNE(n_components=2, learning_rate="auto", perplexity=5)
    X_tsne = tsne.fit_transform(df_features, labels)

    # Plot the t-SNE representation colored by the labels
    sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=labels)
    return plt.show()


def plt_lda(df_features, labels):
    '''
    Using this function will return a 1 dimensional LDA plot. May be used as feature extraction.
    '''

    labels = labels['label'].replace({'benign': 0, 'malignant': 1})
    labels = labels.to_numpy()
    df_features = df_features.to_numpy()

    # Perform LDA
    lda = LDA()
    X_lda = lda.fit_transform(df_features, labels)

    # Plot the t-SNE representation colored by the labels
    plt.scatter(X_lda, np.zeros_like(X_lda), c=labels)
    plt.xlabel('LD1')
    plt.ylim(-0.1, 0.1)

    return plt.show()


def PCA_info_notransform(X, selected_features):
    # Covariance matrix
    covar_matrix = PCA(n_components=len(selected_features))
    covar_matrix.fit(X)

    variance = covar_matrix.explained_variance_ratio_
    var = np.cumsum(np.round(variance, decimals=3)*100)

    return variance, var


def PCA_plot(var):
    plt.ylabel('% Variance Explained')
    plt.xlabel('# of Features')
    plt.title('PCA Variance Explained')
    plt.ylim(min(var), 100.5)
    plt.style.context('seaborn-whitegrid')
    plt.axhline(y=90, color='r', linestyle='--')
    plt.plot(var)
    plt.show()

    for j, val in enumerate(var):
        if val > 90:
            print("The index of the first number that is bigger than 90% is:", j)
            n_comp = j+1
            break
        
    return n_comp


def PCA_transform(X_tr_normalized, n_comp):
    pca = PCA(n_components=n_comp)  # as index starts at 0 the number of components is one higher
    X_pca = pca.fit_transform(X_tr_normalized)

    variance = pca.explained_variance_ratio_
    var = np.cumsum(np.round(variance, decimals=3)*100)

    return pca, X_pca, variance, var


def PCA_transform_overview(variance, var):
    n_components = len(variance)
    idx = np.arange(n_components)+1

    df_explained_variance = pd.DataFrame([variance, var],
                                        index=['explained variance', 'cumulative [%]'],
                                        columns=idx).T

    mean_explained_variance = df_explained_variance.iloc[:, 0].mean()  # calculate mean explained variance

    # DISPLAY info about PCs
    print('PCA Overview')
    print('='*40)
    print("Total: {} components".format(n_components))
    print('-'*40)
    print('Mean explained variance:', round(mean_explained_variance, 3))
    print('-'*40)
    print(df_explained_variance)
    print('-'*40)

    return df_explained_variance, mean_explained_variance, idx


def scree_plot(df_explained_variance, mean_explained_variance, idx):
    limit_df = 15  # change this number if you want to have more or less PCA components visualized
    df_explained_variance_limited = df_explained_variance.iloc[:limit_df, :]

    # make scree plot
    fig, ax1 = plt.subplots(figsize=(15, 6))

    ax1.set_title('Explained variance across principal components', fontsize=14)
    ax1.set_xlabel('Principal component', fontsize=12)
    ax1.set_ylabel('Explained variance', fontsize=12)

    ax2 = sns.barplot(x=idx[:limit_df], y='explained variance', data=df_explained_variance_limited, palette='summer')
    ax2 = ax1.twinx()
    ax2.grid(False)

    ax2.set_ylabel('Cumulative', fontsize=14)
    ax2 = sns.lineplot(x=idx[:limit_df]-1, y='cumulative [%]', data=df_explained_variance_limited, color='#fc8d59')

    ax1.axhline(mean_explained_variance, ls='--', color='#fc8d59')  # plot mean
    ax1.text(-.8, mean_explained_variance+(mean_explained_variance*.05), "average",
             color='#fc8d59', fontsize=14)  # label y axis

    max_y1 = max(df_explained_variance_limited.iloc[:, 0])
    max_y2 = max(df_explained_variance_limited.iloc[:, 1])
    ax1.set(ylim=(0, max_y1+max_y1*.1))
    ax2.set(ylim=(0, max_y2+max_y2*.1))

    return plt.show()


def factor_loadings(X, pca, df_explained_variance):
    # DEFINE PARAMETERS
    selected_features = X.columns
    top_k = 3

    # PCA factor loadings
    df_c = pd.DataFrame(pca.components_, columns=selected_features).T

    print("Factor Loadings for the 1. component \n(explains {0:.2f} of the variance)"
          .format(df_explained_variance.iloc[0, 0]))
    print('='*40, '\n')
    print('Top {} highest'.format(top_k))
    print('-'*40)
    print(df_c.iloc[:, 0].sort_values(ascending=False)[:top_k], '\n')

    print('Top {} lowest'.format(top_k))
    print('-'*40)
    print(df_c.iloc[:, 0].sort_values()[:top_k])

    # Plot heatmap
    size_yaxis = round(X[selected_features].shape[1] * 0.5)

    fig, ax = plt.subplots(figsize=(8, size_yaxis))
    sns.heatmap(df_c.iloc[:, :], annot=True, cmap="YlGnBu", ax=ax)

    return plt.show()


def classifiers(clsfs, X_pca, y_train):
    f1_scores = []
    # Loop over each classifier and fit the model, make predictions and compute metrics
    for clf in clsfs:
        clf.fit(X_pca, y_train)
        y_pred = clf.predict(X_pca)
        accuracy = metrics.accuracy_score(y_train, y_pred)
        F1 = metrics.f1_score(y_train, y_pred)
        f1_scores.append(F1)
        precision = metrics.precision_score(y_train, y_pred)
        recall = metrics.recall_score(y_train, y_pred)
        print(type(clf).__name__)
        print('Accuracy:', accuracy)
        print('F1:', F1)
        print('Precision:', precision)
        print('Recall:', recall)
        print()

    return f1_scores


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [ ]:
# Data loading functions. Uncomment the one you want to use

df = load_data()

# Data clean-up (before splitting data)
In this section double entries, such as repeated features and samples, will be removed from the dataset.

In [ ]:
'''
These 2 lines add a duplicate sample in the DataFrame, you can than observe what the effect is 
in the printed sentence. If you'd like to see it indeed works, please uncomment these lines.
'''
# row = df.iloc[0].copy()
# df = df.append(row, ignore_index=True)
'''
In the lines below we first drop all duplicate entries, whether it is a duplicate row or a column. 
Additionally, the number of duplicates will be returned to the user.
'''

df_f = df.drop_duplicates()
print(f'The number of samples (after removal of duplicates): {len(df_f.index)}, the number of duplicates that had to be removed: {len(df.index)-len(df_f.index)}')
print(f'The number of columns (after removal of duplicates): {len(df_f.columns)}, the number of duplicates that had to be removed: {len(df.columns)-len(df_f.columns)}')

In [ ]:
df = df_f                                       # So we can just use df from now on
df = df.replace('', np.nan)                     # Any empty values are replaced for NaN
df.describe(include='all')                      # Uncomment if interested

check_for_nan = df.isnull().values.any()    # Checking whether the dataset contains of NaN
print(f'Are there NaN existing in the resting dataset?: {check_for_nan}')

non_numeric_columns = df.select_dtypes(exclude='number').columns.tolist()  # Label is a non_numeric_column
numeric_columns = df.select_dtypes(include='number').columns.tolist()
print(f'amount of numeric features = {len(numeric_columns)}')  # All features are numeric_columns

### Conclusion
The table above shows there are only 2 labels present in the dataset, from observation of the csv.file we can see the labels are 'malignant' and 'benigne'. What is also stated here, that 94 of the 186 samples was labeled malignant (and thus 92 samples are labeled benigne.) This information is usefull in the next step, to determine whether a stratified splitting of our data, would be necessary.

# Data splitting
In this section the dataset will be split into two datasets, the Design set (D_set) and the Final Test set (Ft_set). This splitting will be performed randomly where 80% will end up in D_set and the remaining 20% will be stored in Ft_set. Ft_set will not be used at all, until the very latest to evaluate our tool and establish how generalising the tool is. In the initial split we will randomly split the dataset, as both labels are equally abundant in the set.

In [ ]:
# D_set, Ft_set = train_test_split(df, test_size=0.2)
# D_set = pd.DataFrame(D_set)
# Ft_set = pd.DataFrame(Ft_set)                         

Following lines were used to save the splitted sets, to make sure Design and Test set won't alter. 

In [ ]:
# D_set.to_csv("C:/Users/dgjpa/D_set.csv")
# Ft_set = Ft_set.to_csv("C:/Users/dgjpa/Ft_set.csv")

In the lines below, we try to gain some insight in what information our Design dataset contains and how it looks like as a whole.

In [ ]:
D_s = load_D_set()
D_s.head()


In [ ]:
D_s.describe(include='all')

In [ ]:
D_s.info()

Below, we have splitted the Design set into a dataframe including the label, and a dataframe including only the information without label. It can be observed that we have a total of 148 samples in our Design set with a total of 493 features.

In [ ]:
X_d, y_d = return_X_y(D_s)
X_d.shape, y_d.shape

# Data splitting (2/2)
First, there will be two functions who return dataframes from the save .csv file. D_usable and Ft_usable are both DataFrames including both the labels as the feature information, and are the resulting datasets from the initial split as can be seen above.

X_d is a Dataframe including all the information about a case but with the labels removed, whereas y_d is a DataFrame including only the labels.

Also a second split is performed to ensure our preprocessing steps can be validated. The second split should result in a total of 44 files. A total of 11 pairs of validation (ca. 10%) and train sets (ca. 90%), which are consequently splitted into a .csv file containing only feature information (X_Tr/Va) and csv-files containing only the label per sample (y_Tr/Va).

In [ ]:
# # Split Design set into train and validation sets.
# second_split(X_d, y_d)

# Preprocessing
Now that we have splitted our data, we can continue with the preprocessing steps on our training set. It was previously observed there were some zero-variance features present in the Design set. These zero-variances will not contribute in differentiating between any of the sampels and therefore can be savely removed. However, non-zero but small variances are chosen not to be removed based only on their variance as even minor variances could have significant impact in the differentiation between maligne and benigne.

The function below will remove the columns with zero-variance and will return the names of the removed columns, the columns from this Dataframe will be used to remove the same columns in subsequent steps in our model. In other words, based on the zero-variance in specific columns in our training set, these features will not be considered in the validation and test sets as well.

Additionally, the non-zero-variance columns will be screened for fraction of zero or NaN entries. If a column consists of more than 80% of zero's, the column will be removed. Also the names of the specific columns will be returned and used to remove the corresponding columns from validation and test sets as well. 

An overview will be made of the features which still have zero's in them. The count of the zero's will be done and the mean with its std and the mean without the zeros with its std is shown of each feature.

In [ ]:
# X_tr1, y_tr1 = load_Tr_set(1)
# X_tr1.shape, y_tr1.shape

In [ ]:
X_filtered, removed_features_preprocess = preprocessing(X_d) # X_filtered, dataframe containing only columns with a non-zero variance. Features_removed, dataframe containing the names of the removed features.

# zero_count = (X_filtered == 0.0).sum(axis=0)  # Makes a list of the features containing zeros and counts them. 
# features_zero = pd.concat([zero_count, X_filtered.mean(), X_filtered.std(), X_filtered[X_filtered !=0].mean(), X_filtered[X_filtered !=0].std()], axis=1)  # The mean and std will be shown of each feature that contains zero. These are calculated with and without the zeros.
# features_zero.columns = ['zero_count', 'mean', 'std', 'mean_without_0', 'std_without_0']  # Added the names of the columns
# features_zero = features_zero[features_zero['zero_count'] > 0]  # Only shows features that contains one or multiple zeros

Thus a total of 30 features could be removed during the preprocessing step!

# Handeling of Outliers
An outlier was identified as such if an entry was more than three std's separated from the mean of the corresponding feature. The outlier was then replaced with the value of the corresponding lower or upper bound. This method was chosen to preserve the overall distribution of the data as best as possible.

In [ ]:
# Handeling of outliers
X_filtered = outliers(X_filtered)

### Scaling
To give every feature the same weight, all features should be scaled. We choose to do this by min-max scaling. 
Below you find the code to apply this normalization to each column of the dataframe.

In [ ]:
X_norm = X_filtered.apply(normalize_standard_v1)

# # Create a box plot of the original data
# X_filtered.boxplot(figsize=(10,5))
# plt.title('Original Data')
# plt.show()

# # Create a box plot of the normalized data by standard scaling
# X_tr_norm2.boxplot(figsize=(10,5))
# plt.title('Normalized Data Standard')
# plt.show()

In order to use our data we need to transform the labels, and as we have only two labels this is easy and we chose to code benign as 0 and malignant as 1. Additionally we will have to transform both the X and the y Dataframes to a numpy array, in order to use scikit learn modules.

In [ ]:
plt_tsne(X_norm, y_d)

The code beneath was not used

In [ ]:
# plt_lda(X_norm, y_d)

# Feature selection


In [ ]:
from sklearn.preprocessing import LabelEncoder

n_original = X_norm.shape[1]
feature_sizes = [10, 40, 115, n_original]
le = LabelEncoder()
y_d_bin = le.fit_transform(y_d.to_numpy().reshape(-1,))

# classifiers = [LDA(), RandomForestClassifier(), SVC(probability=True)]
# titles = ['LDA', 'RF', 'SVC']

# for i, clf in enumerate(classifiers):
#     plt.subplot(1, 3, i+1)
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('ROC Curve {}'.format(titles[i]))
#     for size in feature_sizes:
#         model = LogisticRegression(max_iter=1000)
#         rfe = RFE(estimator=model, n_features_to_select=size)
#         rfe.fit(X_tr_norm, y_d.to_numpy().reshape(-1,))
#         selected_features = X_tr_norm.columns[rfe.support_]
#         X_tr_norm_selected = X_tr_norm.loc[:, selected_features]
#         clf.fit(X_tr_norm_selected, y_d.to_numpy().reshape(-1,))
#         y_score_selected = clf.predict_proba(X_tr_norm_selected)
#         fpr_selected,tpr_selected,thesholds_selected = roc_curve(y_d_bin, y_score_selected[:, 1])
#         n_selected = X_tr_norm_selected.shape[1]
#         plt.plot(fpr_selected, tpr_selected, label=f'Selected {n_selected} from {n_original} features.')

# plt.legend()
# plt.show()

model = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=model, n_features_to_select=120)
rfe.fit(X_norm, y_d.to_numpy().reshape(-1,))
final_features = X_norm.columns[rfe.support_]
final_X_norm = X_norm.loc[:, final_features]

final_X_norm.boxplot(figsize=(10,5))
plt.title('Normalised data scaled by standard scaling after feature selection by RFE')
plt.show()

## Feature Extraction 
Principal Component Analisys (PCA) is used to reduce the dimensions of this complex problem. It uses the current features to recreate new "features" also called components. These components are each constructed with a combination of the current features.

To determine how many components will be created, it is necessary to get some insight in the data.

In [ ]:
# To get info about whole dataset in PCA
variance1, var1 = PCA_info_notransform(final_X_norm, final_features)
n_comp = PCA_plot(var1)

As is shown above, from index 17 on, the features are sufficient to retrieve above 90% of the variance within the datasets. Therefore it is chosen to perform a PCA analysis with the number of 18 components. A new dataset will be created with 18 components, that are based on the old 40 important features that were retrieved in the features selection.

In [ ]:
# To get the new X_pca dataset
pca, X_pca, variance, var = PCA_transform(final_X_norm, n_comp)
df_explained_variance, mean_explained_variance, idx = PCA_transform_overview(variance, var)
# print(df_explained_variance.shape)
# scree_plot(df_explained_variance, mean_explained_variance, idx) #can be done for every component, now only component 1
# factor_loadings(X_tr_norm_selected, pca, df_explained_variance) #heatmap should be improved after better features selection 

The heatmap above shows for every new component which features are used to create this new component.
NOTE: get this more insightful; how?

## Classifier

It is time to find a model that can make predictions whether a tumor is benign or malign.
There are a lot of different ways of classifying. However we have only so much time and not every classifier makes sense.
To get a quick overview of which classifiers might be helpful for this problem, some general classifiers have been used on the current training data.
After this general research, a few classifiers will be chosen to do tweak the hyperparameters with crossvalidation.

In [ ]:
# Define a list of classifiers
# clsfs = [LinearDiscriminantAnalysis(), QuadraticDiscriminantAnalysis(), GaussianNB(),
#          LogisticRegression(), SGDClassifier(), KNeighborsClassifier(), 
#          RandomForestClassifier(), DecisionTreeClassifier(), svm.SVC()]
# clsfs_strings = [str(c) for c in clsfs]


# y_tr1_bin = y_tr1['label'].replace({'benign': 0, 'malignant': 1})
# y_tr1_bin = y_tr1_bin.to_numpy()

# f1_scores = classifiers(clsfs, X_pca, y_tr1_bin)
# print(f"list of the f1-scores {f1_scores}")

# # Create an empty dataframe with the desired column name
# df_f1_scores = pd.DataFrame(columns=['f1-score'], index=clsfs_strings)

# # Add the f1-scores for each classifier as a row in the dataframe
# for clsfs_str, score in zip(clsfs_strings, f1_scores):
#     df_f1_scores.loc[clsfs_str] = score

# df_f1_scores = df_f1_scores[df_f1_scores['f1-score'] != 1.0] # overfitting is not good so get rid of the 1.0    
# print(df_f1_scores)
# top_k = 5
# df_f1_scores.sort_values(by='f1-score', ascending=False)[:top_k]

In [ ]:
def RF(X_train, X_test, y_train, y_test, n_iter=30):
    best_f1_RF = 0
    best_params_RF = {}
    f1_scores_rf = []
    param_dist = {
        'n_estimators': randint(5,70),
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
        'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    }

    for i in range(n_iter):
        rf = RandomForestClassifier(random_state=i)
        rs = RandomizedSearchCV(
            rf,
            param_distributions=param_dist,
            cv=5,
            n_jobs=-1
        )

        rs.fit(X_train, y_train)
        best_rf = rs.best_estimator_
        y_pred = best_rf.predict(X_test)

        F1 = metrics.f1_score(y_test, y_pred)
        f1_scores_rf.append(F1)

        if F1 > best_f1_RF:
            best_f1_RF = F1
            best_params_RF = rs.best_params_

    print('Best hyperparameters:', best_params_RF)
    print('Best F1 score:', best_f1_RF)
    return best_params_RF, best_f1_RF

X_train, X_test, y_train, y_test = train_test_split(X_pca, y_d_bin, test_size=0.2, random_state=42)

best_params_RF, best_f1_RF = RF(X_train, X_test, y_train, y_test)

RF_best_class = RandomForestClassifier(**best_params_RF)
scores_rf = cross_val_score(RF_best_class, X_pca, y_d_bin, cv=10, scoring='f1_macro')
print("Random forrest plot: the cross validation has a F1-score of %0.2f with a standard deviation of %0.2f" % (scores_rf.mean(), scores_rf.std()))


### After several runs: 
Best hyperparameters: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 20}\
Best F1 score: 0.9600000000000001\
Random forrest plot: the cross validation has a F1-score of 0.81 with a standard deviation of 0.10

In [ ]:
def svm(X_train, X_test, y_train, y_test, n_iter=50):
    best_f1_svm = 0
    best_params_svm = {}
    f1_scores_svm = []
    param_dist_svm = {
        'C': [0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
        'gamma': ['scale', 'auto'] + list(np.logspace(-3, 3, 7))
    }

    for i in range(n_iter):
        rf_svm = SVC(random_state=i)
        rs_svm = RandomizedSearchCV(
            rf_svm,
            param_distributions=param_dist_svm,
            cv=5,
            n_jobs=-1
        )

        rs_svm.fit(X_train, y_train)
        best_rf_svm = rs_svm.best_estimator_
        y_pred_svm = best_rf_svm.predict(X_test)

        F1_svm = metrics.f1_score(y_test, y_pred_svm)
        f1_scores_svm.append(F1_svm)

        if F1_svm > best_f1_svm:
            best_f1_svm = F1_svm
            best_params_svm = rs_svm.best_params_

    print('Best hyperparameters:', best_params_svm)
    print('Best F1 score:', best_f1_svm)
    return best_params_svm, best_f1_svm

X_train, X_test, y_train, y_test = train_test_split(X_pca, y_d_bin, test_size=0.2, random_state=42)
best_params_svm, best_f1_svm = svm(X_train, X_test, y_train, y_test)

svm_best_class = SVC(**best_params_svm)
scores_svm = cross_val_score(svm_best_class, X_pca, y_d_bin, cv=10, scoring='f1_macro')
print("Support Vector Machine: the cross validation has a F1-score of %0.2f with a standard deviation of %0.2f" % (scores_svm.mean(), scores_svm.std()))


### Selected hyperparameters for SVM as input ensembling classifier in final model
Best hyperparameters: {'kernel': 'rbf', 'gamma': 0.001, 'C': 10}\
Best F1 score: 0.888888888888889\
Support Vector Machine: the cross validation has a F1-score of 0.88 with a standard deviation of 0.12

In [ ]:
def lda(X_train, X_test, y_train, y_test, n_iter=6):
    best_f1_lda = 0
    best_params_lda = {}
    f1_scores_lda = []
    n_classes = len(np.unique(y_train))
    param_dist_lda = {
        'store_covariance': [True, False],
        'solver': ['svd', 'lsqr', 'eigen'],
        'priors': [None] + [list(np.random.dirichlet(np.ones(n_classes))) for i in range(10)],
    }

    for i in range(n_iter):
        rf_lda = LinearDiscriminantAnalysis()
        rs_lda = RandomizedSearchCV(
            rf_lda,
            param_distributions=param_dist_lda,
            cv=5,
            n_jobs=-1,
            random_state=i
        )

        rs_lda.fit(X_train, y_train)
        best_rf_lda = rs_lda.best_estimator_
        y_pred_lda = best_rf_lda.predict(X_test)

        F1_lda = metrics.f1_score(y_test, y_pred_lda)
        f1_scores_lda.append(F1_lda)

        if F1_lda > best_f1_lda:
            best_f1_lda = F1_lda
            best_params_lda = rs_lda.best_params_

    print('Best hyperparameters:', best_params_lda)
    print('Best F1 score:', best_f1_lda)
    return best_params_lda, best_f1_lda

X_train, X_test, y_train, y_test = train_test_split(X_pca, y_d_bin, test_size=0.2, random_state=42)
best_params_lda, best_f1_lda = lda(X_train, X_test, y_train, y_test)


lda_best_class = LinearDiscriminantAnalysis(**best_params_lda)
scores_lda = cross_val_score(lda_best_class, X_pca, y_d_bin, cv=10, scoring='f1_macro')
print("Linear Discriminant Analysis: the cross validation has a F1-score of %0.2f with a standard deviation of %0.2f" % (scores_lda.mean(), scores_lda.std()))


### Selected hyperparameters for LDA as input ensembling classifier in final model
Best hyperparameters: {'store_covariance': True, 'solver': 'eigen', 'priors': [0.5287484663893824, 0.4712515336106177]}\
Best F1 score: 0.7857142857142856\
Linear Discriminant Analysis: the cross validation has a F1-score of 0.91 with a standard deviation of 0.14

## Ensembling
Next ensemble will be used to look if the several classifiers can compromise for each weaknesses and then in the end get a better result. Ensembling is a whole topic on its own, and therefore it is choosen to only do one ensemble. The three classifiers with the best hyperparameters will be used to create one classifiers that uses them all. By cross validation this new classifier will be compared to each one on its own.

In [ ]:
# Create an ensemble of bagged classifiers
svm_bag = BaggingClassifier(estimator=svm_best_class, n_estimators=10, random_state=42)
lda_bag = BaggingClassifier(estimator=lda_best_class, n_estimators=10, random_state=42)
rf_bag = BaggingClassifier(estimator=RF_best_class, n_estimators=10, random_state=42)

ensemble = VotingClassifier(estimators=[('svm', svm_bag), ('lda', lda_bag), ('rf', rf_bag)], voting='hard')

# Perform cross-validation on the ensemble model
scores_ensemble = cross_val_score(ensemble, X_pca, y_d_bin, cv=10, scoring='f1_macro')

# Print the mean and standard deviation of the F1 scores
print("Ensemble: the cross-validation has a mean F1 score of %0.2f with a standard deviation of %0.2f" % (scores_ensemble.mean(), scores_ensemble.std()))

# Start pipeline

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.preprocessing import FunctionTransformer

Ft_s = load_Ft_set()
D_s = load_D_set()

X_ft, y_ft = return_X_y(Ft_s)
X_d, y_d = return_X_y(D_s)

le = LabelEncoder()
y_d_bin = le.fit_transform(y_d.to_numpy().reshape(-1,))
y_ft_bin = le.fit_transform(y_ft.to_numpy().reshape(-1,))

# Parameters needed: removed_features_preprocess, final_features, n_comp, **rf_hyperparams, **svm_hyperparams, **lda_hyperparams
X_filtered, removed_features_preprocess = preprocessing(X_d)
final_features

rf_hyperparams = {
    'max_depth': 20,
    'min_samples_leaf': 2,
    'min_samples_split': 6,
    'n_estimators': 20
}

svm_hyperparams = {
    'kernel': 'rbf',
    'gamma': 0.001,
    'C': 10
}

lda_hyperparams = {
    'store_covariance': False,
    'solver': 'svd',
    'priors': [0.6231810575673354, 0.3768189424326645]
}


def remove_features(X, removed_features_preprocess):
    # implementation of feature removal function
    return X.drop(columns=removed_features_preprocess)


def outliers(X):
    X_clean = X.copy()

    means = X.mean()
    stds = X.std()
    threshold = 3
    for feature in X.columns:

        # Calculate the lower and upper bounds for the feature
        lower_bound = means[feature] - (stds[feature] * threshold)
        upper_bound = means[feature] + (stds[feature] * threshold)

        # Replace outliers with upper and lower bound
        X_clean.loc[X_clean[feature] < lower_bound, feature] = lower_bound
        X_clean.loc[X_clean[feature] > upper_bound, feature] = upper_bound
    
    return X_clean


def normalize_standard(X):
    # apply StandardScaler to each column of X
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    # create a new DataFrame with scaled data
    X_normalized = pd.DataFrame(X_scaled, columns=X.columns)

    return X_normalized


def feature_selection(X, final_features):
    return X[final_features]


# Define the pipeline
pipeline = Pipeline([
    ('remove_features', FunctionTransformer(remove_features, kw_args={'removed_features_preprocess': removed_features_preprocess})),
    ('outliers', FunctionTransformer(outliers)),
    ('normalize', FunctionTransformer(normalize_standard)),
    ('feature_selection', FunctionTransformer(feature_selection, kw_args={'final_features': final_features})),
    ('pca', PCA(n_components=n_comp)),
    ('bagging', BaggingClassifier(
        estimator=VotingClassifier([
            ('rf', RandomForestClassifier(**rf_hyperparams)),
            ('svm', SVC(**svm_hyperparams)),
            ('lda', LinearDiscriminantAnalysis(**lda_hyperparams))
        ], voting='hard'),
        n_estimators=10,
        random_state=42,
        n_jobs=-1
    ))
])

# Train the pipeline
pipeline.fit(X_d, y_d_bin)

# Predict using the pipeline
y_pred = pipeline.predict(X_ft)

f1_macro = f1_score(y_ft_bin, y_pred, average='macro')
print('The F1 score of the pipeline on the test set is:', f1_macro)